# MAK DRAFT HELPER 2019

In [290]:
import pandas as pd

## Batters

In [291]:
batters = pd.read_csv('./FantasyPros_2019_Projections_H.csv')

Fantasy Pros (fantasypros.com) produces an aggregated expert projections for each player across several key stats. No need to reinvent the wheel like Nate Silver did in his book with inefficient models - we just take what the experts project (using their models) and look at their consensus.

In [292]:
batters.head()

,Player,Team,Positions,AB,R,HR,RBI,SB,AVG,OBP,H,2B,3B,BB,SO,SLG,OPS,Own
0,Mike Trout,LAA,"CF,DH",509.0,113.0,39.0,97.0,24.0,0.307,0.440,156.0,29.0,4.0,121.0,129.0,0.612,1.052,91%
1,Mookie Betts,BOS,"CF,RF",591.0,117.0,30.0,93.0,27.0,0.307,0.387,182.0,43.0,4.0,77.0,94.0,0.547,0.934,92%
2,Jose Ramirez,CLE,"2B,3B",571.0,101.0,30.0,97.0,25.0,0.287,0.376,164.0,41.0,4.0,81.0,77.0,0.536,0.912,91%
3,Nolan Arenado,COL,3B,588.0,99.0,37.0,113.0,2.0,0.292,0.363,172.0,38.0,4.0,65.0,115.0,0.560,0.923,91%
4,J.D. Martinez,BOS,"LF,RF,DH",547.0,98.0,40.0,116.0,4.0,0.305,0.378,167.0,34.0,2.0,64.0,150.0,0.591,0.969,91%


In [293]:
del batters['OBP']
del batters['2B']
del batters['3B']
del batters['SLG']
del batters['OPS']

In [294]:
batters = batters[batters['AB'] >= 200]

In [295]:
batters['BB/KO'] = batters['BB'] / batters['SO']

For BB/KO, we want to look at the number of walks ABOVE the average BB/KO of the selection set (which is all batters projected to have over 200 AB's). This will approproately weight the average based on projected Fantasy production. We do the same with batting average.

In [296]:
bbko_mean = batters['BB/KO'].mean()
bbko_mean

0.3966996500649177

In [297]:
batters['BB+'] = (batters['BB/KO'] - bbko_mean) * batters['SO']

In [298]:
batters['AVG'] = batters['H'] / batters['AB']

In [299]:
avg_mean = batters['AVG'].mean()
avg_mean

0.2529530803912304

In [300]:
batters['AVG+'] = (batters['AVG'] - avg_mean) * batters['AB']

For this year, I'm simply standardizing each metric to the 0-100 linear scale with 100 being the top performer for that metric and 0 being the bottom. Since the scales are different for each one, this makes sense. In the past, I've used Z scores, but it is easier to be able to mentally calculate the actual metric in some cases. It also makes sense since these stats all accumulate over the course of the season/week.

In [301]:
batters['AVGS'] = round( ( batters['AVG+'] - batters['AVG+'].min() ) / ( batters['AVG+'].max() - batters['AVG+'].min() ) * 100 )
batters['BBKOS'] = round( ( batters['BB+'] - batters['BB+'].min() ) / ( batters['BB+'].max() - batters['BB+'].min() ) * 100 )
batters['HRS'] = round( ( batters['HR'] - batters['HR'].min() ) / ( batters['HR'].max() - batters['HR'].min() ) * 100 )
batters['RS'] = round( ( batters['R'] - batters['R'].min() ) / ( batters['R'].max() - batters['R'].min() ) * 100 )
batters['RBIS'] = round( ( batters['RBI'] - batters['RBI'].min() ) / ( batters['RBI'].max() - batters['RBI'].min() ) * 100 )
batters['SBS'] = round( ( batters['SB'] - batters['SB'].min() ) / ( batters['SB'].max() - batters['SB'].min() ) * 100 )
batters['SCORE'] = batters['AVGS'] + batters['BBKOS'] + batters['HRS'] + batters['RS'] + batters['RBIS'] + batters['SBS']
batters['RANK'] = batters['SCORE'].rank(method='average', na_option='keep', ascending=False, pct=False)

In [302]:
batters.sort_values(['SCORE'], ascending=[0])[0:5]

,Player,Team,Positions,AB,R,HR,RBI,SB,AVG,H,...,BB+,AVG+,AVGS,BBKOS,HRS,RS,RBIS,SBS,SCORE,RANK
0,Mike Trout,LAA,"CF,DH",509.0,113.0,39.0,97.0,24.0,0.306483,156.0,...,69.825745,27.246882,85.0,100.0,90.0,96.0,81.0,52.0,504.0,1.0
1,Mookie Betts,BOS,"CF,RF",591.0,117.0,30.0,93.0,27.0,0.307953,182.0,...,39.710233,32.504729,94.0,77.0,69.0,100.0,77.0,59.0,476.0,2.0
2,Jose Ramirez,CLE,"2B,3B",571.0,101.0,30.0,97.0,25.0,0.287215,164.0,...,50.454127,19.563791,73.0,85.0,69.0,84.0,81.0,54.0,446.0,3.0
4,J.D. Martinez,BOS,"LF,RF,DH",547.0,98.0,40.0,116.0,4.0,0.305302,167.0,...,4.495052,28.634665,88.0,50.0,93.0,80.0,100.0,9.0,420.0,4.0
5,Christian Yelich,MIL,"LF,CF,RF",574.0,102.0,28.0,93.0,17.0,0.299652,172.0,...,19.048848,26.804932,85.0,61.0,64.0,85.0,77.0,37.0,409.0,5.0


## Pitchers

In [303]:
pitchers = pd.read_csv('./FantasyPros_2019_Projections_P.csv')

In [304]:
pitchers.head()

,Player,Team,Positions,IP,K,W,SV,ERA,WHIP,ER,H,BB,HR,G,GS,L,CG,Own
0,Max Scherzer,WSH,SP,207.6,269.0,16.0,0.0,2.90,1.00,67.0,154.0,54.0,26.0,32.0,32.0,8.0,2.0,91%
1,Chris Sale,BOS,SP,185.0,249.0,15.0,0.0,2.61,0.97,54.0,139.0,40.0,18.0,29.0,29.0,6.0,0.0,91%
2,Jacob deGrom,NYM,SP,207.8,248.0,15.0,0.0,2.71,1.05,63.0,167.0,51.0,19.0,32.0,32.0,8.0,1.0,91%
3,Justin Verlander,HOU,SP,200.8,244.0,15.0,0.0,3.15,1.05,70.0,160.0,51.0,27.0,32.0,31.0,8.0,1.0,91%
4,Corey Kluber,CLE,SP,207.1,220.0,16.0,0.0,3.22,1.07,74.0,180.0,42.0,24.0,32.0,32.0,8.0,3.0,91%


In [305]:
del pitchers['HR']
del pitchers['G']
del pitchers['GS']
del pitchers['CG']

In [306]:
pitchers = pitchers[pitchers['IP'] >= 40]

In [307]:
pitchers['ERA'] = pitchers['ER'] / pitchers['IP'] * 9
era_mean = pitchers['ERA'].mean()
pitchers['ERA+'] = ( era_mean - pitchers['ERA'] ) * pitchers['IP']

In [308]:
pitchers['WHIP'] = (pitchers['BB'] + pitchers['H']) / pitchers['IP']
whip_mean = pitchers['WHIP'].mean()
pitchers['WHIP+'] = ( whip_mean -  pitchers['WHIP'] ) * pitchers['IP']

In [309]:
pitchers['W-L'] = pitchers['W'] - pitchers['L']

In [310]:
pitchers['ERAS'] = round( ( pitchers['ERA+'] - pitchers['ERA+'].min() ) / ( pitchers['ERA+'].max() - pitchers['ERA+'].min() ) * 100 )
pitchers['WHIPS'] = round( ( pitchers['WHIP+'] - pitchers['WHIP+'].min() ) / ( pitchers['WHIP+'].max() - pitchers['WHIP+'].min() ) * 100 )
pitchers['KS'] = round( ( pitchers['K'] - pitchers['K'].min() ) / ( pitchers['K'].max() - pitchers['K'].min() ) * 100 )
pitchers['SVS'] = round( ( pitchers['SV'] - pitchers['SV'].min() ) / ( pitchers['SV'].max() - pitchers['SV'].min() ) * 100 )
pitchers['WLS'] = round( ( pitchers['W-L'] - pitchers['W-L'].min() ) / ( pitchers['W-L'].max() - pitchers['W-L'].min() ) * 100 )
pitchers['SCORE'] = pitchers['ERAS'] + pitchers['WHIPS'] + pitchers['KS'] + pitchers['SVS'] + pitchers['WLS']
pitchers['RANK'] = pitchers['SCORE'].rank(method='average', na_option='keep', ascending=False, pct=False)

In [311]:
pitchers.sort_values(['SCORE'], ascending=[0])[0:5]

,Player,Team,Positions,IP,K,W,SV,ERA,WHIP,ER,...,ERA+,WHIP+,W-L,ERAS,WHIPS,KS,SVS,WLS,SCORE,RANK
0,Max Scherzer,WSH,SP,207.6,269.0,16.0,0.0,2.904624,1.001927,67.0,...,305.430616,78.916711,8.0,94.0,100.0,100.0,0.0,93.0,387.0,1.5
1,Chris Sale,BOS,SP,185.0,249.0,15.0,0.0,2.627027,0.967568,54.0,...,323.535954,76.682040,9.0,97.0,98.0,92.0,0.0,100.0,387.0,1.5
2,Jacob deGrom,NYM,SP,207.8,248.0,15.0,0.0,2.728585,1.049086,63.0,...,342.305790,69.193124,7.0,100.0,93.0,92.0,0.0,86.0,371.0,3.0
3,Justin Verlander,HOU,SP,200.8,244.0,15.0,0.0,3.137450,1.050797,70.0,...,248.674700,66.518668,7.0,85.0,91.0,90.0,0.0,86.0,352.0,4.0
4,Corey Kluber,CLE,SP,207.1,220.0,16.0,0.0,3.215838,1.071946,74.0,...,240.242681,64.225678,8.0,84.0,90.0,80.0,0.0,93.0,347.0,5.0


## Combining Pitchers and Batters

Since we score across five pitching and six hitting categories, with the weekly winners determined by the highest aggregate scores in each category, it makes sense to compare score weight of batters vs. pitchers 

For the last couple of seasons, I've tried to "break the game" by focusing only on all the pitching categories plus two batting categories (AVG and BBKO) where there are batters generally available with high performance at a low cost. This has had mixed results, as the league adjusted accordingly.

In [312]:
batters_lim = batters[ ['Player', 'Team', 'Positions', 'AVGS', 'BBKOS', 'HRS', 'RS', 'RBIS', 'SBS', 'SCORE'] ]
pitchers_lim = pitchers[ ['Player', 'Team', 'Positions', 'ERAS', 'WHIPS', 'KS', 'SVS', 'WLS', 'SCORE'] ]
all_players = pd.concat([batters_lim, pitchers_lim],sort=True)
all_players['RANK'] = all_players['SCORE'].rank(method='average', na_option='keep', ascending=False, pct=False)
all_players = all_players.sort_values(['SCORE'], ascending=[0])

In [313]:
all_players.head(25)

,AVGS,BBKOS,ERAS,HRS,KS,Player,Positions,RBIS,RS,SBS,SCORE,SVS,Team,WHIPS,WLS,RANK
0,85.0,100.0,NaN,90.0,NaN,Mike Trout,"CF,DH",81.0,96.0,52.0,504.0,NaN,LAA,NaN,NaN,1.0
1,94.0,77.0,NaN,69.0,NaN,Mookie Betts,"CF,RF",77.0,100.0,59.0,476.0,NaN,BOS,NaN,NaN,2.0
2,73.0,85.0,NaN,69.0,NaN,Jose Ramirez,"2B,3B",81.0,84.0,54.0,446.0,NaN,CLE,NaN,NaN,3.0
4,88.0,50.0,NaN,93.0,NaN,J.D. Martinez,"LF,RF,DH",100.0,80.0,9.0,420.0,NaN,BOS,NaN,NaN,4.0
5,85.0,61.0,NaN,64.0,NaN,Christian Yelich,"LF,CF,RF",77.0,85.0,37.0,409.0,NaN,MIL,NaN,NaN,5.0
3,79.0,61.0,NaN,86.0,NaN,Nolan Arenado,3B,97.0,81.0,4.0,408.0,NaN,COL,NaN,NaN,6.5
10,53.0,88.0,NaN,79.0,NaN,Bryce Harper,"CF,RF",83.0,81.0,24.0,408.0,NaN,PHI,NaN,NaN,6.5
8,70.0,65.0,NaN,67.0,NaN,Francisco Lindor,SS,70.0,84.0,41.0,397.0,NaN,CLE,NaN,NaN,8.5
12,71.0,80.0,NaN,64.0,NaN,Alex Bregman,"3B,SS",78.0,80.0,24.0,397.0,NaN,HOU,NaN,NaN,8.5
9,100.0,65.0,NaN,40.0,NaN,Jose Altuve,2B,63.0,77.0,48.0,393.0,NaN,HOU,NaN,NaN,10.0


## Join on list of currently drafted players

In [314]:
draft = pd.read_csv('./results.csv', header=1)
draft.head()

,Pick,Team,Player,Elapsed Time,Unnamed: 4
0,1,MitchMoreland'sBattingAverage,Mike Trout CF | LAA,NaN,NaN
1,2,Tommy's Shinebox,Mookie Betts RF | BOS,8 hrs 29 min 27 sec,NaN
2,3,La Flama Blanca,Jose Altuve 2B | HOU,32 min 24 sec,NaN
3,4,Low hanging fruit,Max Scherzer SP | WAS,2 min 59 sec,NaN
4,5,Foul_Tip,Nolan Arenado 3B | COL,57 min 17 sec,NaN


In [315]:
draft = draft[ ['Player', 'Team'] ]
draft['Name'] = draft['Player']
draft['Owner'] = draft['Team']
draft = draft[ draft['Player'] != 'Player' ]
draft['Player'] = draft['Name'].str.extract(r'^(.+ .+) \w+ \|', expand=False)
draft['Team'] = draft['Name'].str.extract(r'\|\s(\w\w\w|\w\w)(?:\s|$)', expand=False)
draft = draft.dropna()

In [316]:
draft = draft.replace('WAS', 'WSH')

In [317]:
draft.head()

,Player,Team,Name,Owner
0,Mike Trout,LAA,Mike Trout CF | LAA,MitchMoreland'sBattingAverage
1,Mookie Betts,BOS,Mookie Betts RF | BOS,Tommy's Shinebox
2,Jose Altuve,HOU,Jose Altuve 2B | HOU,La Flama Blanca
3,Max Scherzer,WSH,Max Scherzer SP | WAS,Low hanging fruit
4,Nolan Arenado,COL,Nolan Arenado 3B | COL,Foul_Tip


In [318]:
all_players_owners = all_players.merge(draft, on=['Player', 'Team'], how='outer')

In [319]:
all_players_owners.head()

,AVGS,BBKOS,ERAS,HRS,KS,Player,Positions,RBIS,RS,SBS,SCORE,SVS,Team,WHIPS,WLS,RANK,Name,Owner
0,85.0,100.0,NaN,90.0,NaN,Mike Trout,"CF,DH",81.0,96.0,52.0,504.0,NaN,LAA,NaN,NaN,1.0,Mike Trout CF | LAA,MitchMoreland'sBattingAverage
1,94.0,77.0,NaN,69.0,NaN,Mookie Betts,"CF,RF",77.0,100.0,59.0,476.0,NaN,BOS,NaN,NaN,2.0,Mookie Betts RF | BOS,Tommy's Shinebox
2,73.0,85.0,NaN,69.0,NaN,Jose Ramirez,"2B,3B",81.0,84.0,54.0,446.0,NaN,CLE,NaN,NaN,3.0,Jose Ramirez 3B | CLE (Keeper),BC-Mizusawa South Monkeyz
3,88.0,50.0,NaN,93.0,NaN,J.D. Martinez,"LF,RF,DH",100.0,80.0,9.0,420.0,NaN,BOS,NaN,NaN,4.0,J.D. Martinez DH | BOS,stroke me strokeme
4,85.0,61.0,NaN,64.0,NaN,Christian Yelich,"LF,CF,RF",77.0,85.0,37.0,409.0,NaN,MIL,NaN,NaN,5.0,Christian Yelich LF | MIL,BC-Mizusawa South Monkeyz


Ensure all players merged. Cross fingers that there aren't any players on the same team with the same name. That would be weird though. This takes advatage of the characterisitics of an outer join.

In [320]:
all_players_owners[all_players_owners['Player'].isnull()].head()

,AVGS,BBKOS,ERAS,HRS,KS,Player,Positions,RBIS,RS,SBS,SCORE,SVS,Team,WHIPS,WLS,RANK,Name,Owner


In [321]:
all_players_owners[all_players_owners['Positions'].isnull()].head()

,AVGS,BBKOS,ERAS,HRS,KS,Player,Positions,RBIS,RS,SBS,SCORE,SVS,Team,WHIPS,WLS,RANK,Name,Owner


## All Currently Undrafted

In [332]:
all_players_owners = all_players_owners[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SVS', 'Owner'] ]
free_agents = all_players_owners[all_players_owners['Owner'].isnull()]
del free_agents['Owner']
free_agents.head(50)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,ERAS,WHIPS,KS,WLS,SVS
26,Joey Votto,1B,CIN,357.0,27.0,72.0,97.0,50.0,65.0,66.0,7.0,NaN,NaN,NaN,NaN,NaN
33,Xander Bogaerts,SS,BOS,341.0,33.5,72.0,56.0,45.0,75.0,69.0,24.0,NaN,NaN,NaN,NaN,NaN
34,Nelson Cruz,DH,MIN,340.0,35.0,59.0,50.0,81.0,85.0,63.0,2.0,NaN,NaN,NaN,NaN,NaN
35,George Springer,"CF,RF,DH",HOU,338.0,36.5,54.0,60.0,62.0,63.0,82.0,17.0,NaN,NaN,NaN,NaN,NaN
38,Starling Marte,CF,PIT,335.0,39.0,67.0,38.0,40.0,53.0,63.0,74.0,NaN,NaN,NaN,NaN,NaN
39,Matt Carpenter,"1B,2B,3B",STL,329.0,40.0,42.0,77.0,64.0,61.0,78.0,7.0,NaN,NaN,NaN,NaN,NaN
40,Yasiel Puig,RF,CIN,327.0,41.0,57.0,53.0,62.0,65.0,57.0,33.0,NaN,NaN,NaN,NaN,NaN
41,Lorenzo Cain,CF,MIL,326.0,42.0,75.0,62.0,29.0,38.0,68.0,54.0,NaN,NaN,NaN,NaN,NaN
42,Daniel Murphy,"1B,2B",COL,324.0,43.5,86.0,56.0,48.0,65.0,60.0,9.0,NaN,NaN,NaN,NaN,NaN
46,Jean Segura,SS,PHI,321.0,47.0,77.0,48.0,29.0,49.0,70.0,48.0,NaN,NaN,NaN,NaN,NaN


## Positions

### 1B

In [335]:
first_base = all_players_owners[all_players_owners['Positions'].str.contains('1B')]
first_base = first_base[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [336]:
first_base.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
14,Paul Goldschmidt,1B,STL,386.0,15.0,70.0,69.0,69.0,78.0,78.0,22.0,Martin Awaiting Klub
15,Freddie Freeman,1B,ATL,381.0,16.0,85.0,66.0,62.0,77.0,74.0,17.0,Walk Off
20,Anthony Rizzo,1B,CHC,374.0,21.0,65.0,78.0,64.0,82.0,70.0,15.0,Low hanging fruit
24,Rhys Hoskins,"1B,LF",PHI,361.0,25.0,39.0,66.0,83.0,88.0,74.0,11.0,La Flama Blanca
26,Joey Votto,1B,CIN,357.0,27.0,72.0,97.0,50.0,65.0,66.0,7.0,NaN
31,Cody Bellinger,"1B,CF",LAD,343.0,32.0,47.0,54.0,71.0,76.0,69.0,26.0,Foul_Tip
37,Whit Merrifield,"1B,2B,CF,RF,DH",KC,337.0,38.0,71.0,51.0,29.0,48.0,64.0,74.0,Regular Rubens
39,Matt Carpenter,"1B,2B,3B",STL,329.0,40.0,42.0,77.0,64.0,61.0,78.0,7.0,NaN
42,Daniel Murphy,"1B,2B",COL,324.0,43.5,86.0,56.0,48.0,65.0,60.0,9.0,NaN
50,Carlos Santana,"1B,3B",CLE,320.0,49.5,36.0,93.0,52.0,68.0,64.0,7.0,NaN


### 2B

In [337]:
second_base = all_players_owners[all_players_owners['Positions'].str.contains('2B')]
second_base = second_base[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [338]:
second_base.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
2,Jose Ramirez,"2B,3B",CLE,446.0,3.0,73.0,85.0,69.0,81.0,84.0,54.0,BC-Mizusawa South Monkeyz
9,Jose Altuve,2B,HOU,393.0,10.0,100.0,65.0,40.0,63.0,77.0,48.0,La Flama Blanca
32,Javier Baez,"2B,3B,SS",CHC,341.0,33.5,63.0,22.0,69.0,81.0,69.0,37.0,Team Gorgeous
37,Whit Merrifield,"1B,2B,CF,RF,DH",KC,337.0,38.0,71.0,51.0,29.0,48.0,64.0,74.0,Regular Rubens
39,Matt Carpenter,"1B,2B,3B",STL,329.0,40.0,42.0,77.0,64.0,61.0,78.0,7.0,NaN
42,Daniel Murphy,"1B,2B",COL,324.0,43.5,86.0,56.0,48.0,65.0,60.0,9.0,NaN
58,Adalberto Mondesi,"2B,SS",KC,311.0,59.5,38.0,23.0,45.0,52.0,57.0,96.0,Foul_Tip
60,Travis Shaw,"1B,3B,2B",MIL,310.0,62.5,40.0,59.0,67.0,73.0,58.0,13.0,NaN
63,Ozzie Albies,2B,ATL,310.0,62.5,57.0,44.0,45.0,58.0,71.0,35.0,NaN
74,Gleyber Torres,"2B,SS",NYY,299.0,75.5,54.0,44.0,57.0,70.0,54.0,20.0,NaN


### SS

In [345]:
shortstop = all_players_owners[all_players_owners['Positions'].str.contains('SS')]
shortstop = shortstop[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [346]:
shortstop.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
7,Francisco Lindor,SS,CLE,397.0,8.5,70.0,65.0,67.0,70.0,84.0,41.0,Walk Off
8,Alex Bregman,"3B,SS",HOU,397.0,8.5,71.0,80.0,64.0,78.0,80.0,24.0,Martin Awaiting Klub
10,Trea Turner,SS,WSH,391.0,11.0,68.0,51.0,40.0,54.0,78.0,100.0,Martin Awaiting Klub
16,Manny Machado,"3B,SS",SD,378.0,17.0,72.0,60.0,76.0,80.0,68.0,22.0,MitchMoreland'sBattingAverage
25,Trevor Story,SS,COL,360.0,26.0,60.0,32.0,76.0,82.0,67.0,43.0,Team Gorgeous
32,Javier Baez,"2B,3B,SS",CHC,341.0,33.5,63.0,22.0,69.0,81.0,69.0,37.0,Team Gorgeous
33,Xander Bogaerts,SS,BOS,341.0,33.5,72.0,56.0,45.0,75.0,69.0,24.0,NaN
46,Jean Segura,SS,PHI,321.0,47.0,77.0,48.0,29.0,49.0,70.0,48.0,NaN
49,Carlos Correa,SS,HOU,320.0,49.5,55.0,59.0,55.0,75.0,65.0,11.0,NaN
58,Adalberto Mondesi,"2B,SS",KC,311.0,59.5,38.0,23.0,45.0,52.0,57.0,96.0,Foul_Tip


### 3B

In [349]:
third_base = all_players_owners[all_players_owners['Positions'].str.contains('3B')]
third_base = third_base[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [350]:
third_base.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
2,Jose Ramirez,"2B,3B",CLE,446.0,3.0,73.0,85.0,69.0,81.0,84.0,54.0,BC-Mizusawa South Monkeyz
5,Nolan Arenado,3B,COL,408.0,6.5,79.0,61.0,86.0,97.0,81.0,4.0,Foul_Tip
8,Alex Bregman,"3B,SS",HOU,397.0,8.5,71.0,80.0,64.0,78.0,80.0,24.0,Martin Awaiting Klub
16,Manny Machado,"3B,SS",SD,378.0,17.0,72.0,60.0,76.0,80.0,68.0,22.0,MitchMoreland'sBattingAverage
27,Anthony Rendon,3B,WSH,353.0,28.0,75.0,69.0,55.0,75.0,68.0,11.0,stroke me strokeme
30,Kris Bryant,"3B,RF",CHC,347.0,30.5,63.0,62.0,62.0,69.0,78.0,13.0,Rakuten Golden Eagles
32,Javier Baez,"2B,3B,SS",CHC,341.0,33.5,63.0,22.0,69.0,81.0,69.0,37.0,Team Gorgeous
39,Matt Carpenter,"1B,2B,3B",STL,329.0,40.0,42.0,77.0,64.0,61.0,78.0,7.0,NaN
44,Eugenio Suarez,3B,CIN,323.0,45.5,54.0,53.0,69.0,77.0,63.0,7.0,Low hanging fruit
50,Carlos Santana,"1B,3B",CLE,320.0,49.5,36.0,93.0,52.0,68.0,64.0,7.0,NaN


### C

In [351]:
catcher = all_players_owners[all_players_owners['Positions'].str.contains('^C$|C,', regex=True)]
catcher = catcher[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [352]:
catcher.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
108,J.T. Realmuto,"C,1B",PHI,277.0,109.0,59.0,43.0,50.0,60.0,54.0,11.0,Team Gorgeous
118,Gary Sanchez,"C,DH",NYY,272.0,116.5,35.0,45.0,67.0,66.0,55.0,4.0,NaN
178,Buster Posey,"C,1B",SF,243.0,180.0,68.0,66.0,21.0,42.0,39.0,7.0,NaN
207,Yadier Molina,C,STL,234.0,209.0,53.0,48.0,36.0,51.0,35.0,11.0,NaN
223,Yasmani Grandal,C,MIL,228.0,225.0,32.0,57.0,50.0,49.0,38.0,2.0,Martin Awaiting Klub
242,Willson Contreras,C,CHC,220.0,243.0,44.0,51.0,33.0,47.0,36.0,9.0,Foul_Tip
288,Wilson Ramos,"C,DH",NYM,203.0,290.5,53.0,45.0,36.0,45.0,24.0,0.0,NaN
337,Francisco Cervelli,C,PIT,190.0,335.5,45.0,58.0,21.0,33.0,26.0,7.0,NaN
348,Danny Jansen,C,TOR,185.0,350.0,39.0,53.0,26.0,32.0,28.0,7.0,NaN
388,Tucker Barnhart,C,CIN,175.0,388.0,40.0,56.0,19.0,31.0,25.0,4.0,NaN


### OF

In [353]:
outfield = all_players_owners[all_players_owners['Positions'].str.contains('OF|LF|RF|CF', regex=True)]
outfield = outfield[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [360]:
outfield[:24]

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
0,Mike Trout,"CF,DH",LAA,504.0,1.0,85.0,100.0,90.0,81.0,96.0,52.0,MitchMoreland'sBattingAverage
1,Mookie Betts,"CF,RF",BOS,476.0,2.0,94.0,77.0,69.0,77.0,100.0,59.0,Tommy's Shinebox
3,J.D. Martinez,"LF,RF,DH",BOS,420.0,4.0,88.0,50.0,93.0,100.0,80.0,9.0,stroke me strokeme
4,Christian Yelich,"LF,CF,RF",MIL,409.0,5.0,85.0,61.0,64.0,77.0,85.0,37.0,BC-Mizusawa South Monkeyz
6,Bryce Harper,"CF,RF",PHI,408.0,6.5,53.0,88.0,79.0,83.0,81.0,24.0,Regular Rubens
11,Juan Soto,LF,WSH,388.0,12.0,73.0,79.0,67.0,78.0,76.0,15.0,Rakuten Golden Eagles
17,Giancarlo Stanton,"LF,RF,DH",NYY,377.0,18.0,54.0,44.0,100.0,93.0,79.0,7.0,Tommy's Shinebox
18,Aaron Judge,"RF,DH",NYY,376.0,19.0,52.0,62.0,88.0,78.0,81.0,15.0,Foul_Tip
19,Ronald Acuna,"LF,CF",ATL,375.0,20.0,65.0,42.0,69.0,71.0,78.0,50.0,Team Gorgeous
21,Charlie Blackmon,CF,COL,373.0,22.5,81.0,51.0,64.0,62.0,89.0,26.0,La Flama Blanca


In [361]:
outfield[25:49]

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
55,Eddie Rosario,LF,MIN,313.0,56.0,66.0,36.0,57.0,67.0,67.0,20.0,NaN
56,Mitch Haniger,"CF,RF",SEA,312.0,57.5,57.0,51.0,55.0,69.0,65.0,15.0,NaN
66,Nicholas Castellanos,RF,DET,308.0,66.0,69.0,39.0,57.0,74.0,62.0,7.0,MitchMoreland'sBattingAverage
68,Justin Upton,"LF,DH",LAA,307.0,68.5,38.0,42.0,69.0,74.0,64.0,20.0,NaN
69,Michael Conforto,"LF,CF,RF",NYM,305.0,70.0,39.0,59.0,67.0,70.0,63.0,7.0,NaN
72,Joey Gallo,"1B,LF,CF,RF",TEX,302.0,72.5,11.0,43.0,93.0,77.0,67.0,11.0,NaN
76,David Peralta,LF,ARI,298.0,77.0,68.0,47.0,52.0,61.0,59.0,11.0,NaN
82,A.J. Pollock,CF,LAD,294.0,83.5,47.0,46.0,50.0,55.0,59.0,37.0,NaN
83,Ender Inciarte,CF,ATL,294.0,83.5,64.0,57.0,21.0,42.0,60.0,50.0,NaN
92,Wil Myers,"3B,LF,RF",SD,286.0,94.0,38.0,42.0,52.0,57.0,56.0,41.0,BC-Mizusawa South Monkeyz


In [364]:
#outfield[50:74]

### SP

In [365]:
starter = all_players_owners[all_players_owners['Positions'].str.contains('SP')]
starter = starter[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SVS', 'Owner'] ]

In [366]:
starter[:24]

,Player,Positions,Team,SCORE,RANK,ERAS,WHIPS,KS,WLS,SVS,Owner
12,Chris Sale,SP,BOS,387.0,13.5,97.0,98.0,92.0,100.0,0.0,Rakuten Golden Eagles
13,Max Scherzer,SP,WSH,387.0,13.5,94.0,100.0,100.0,93.0,0.0,Low hanging fruit
23,Jacob deGrom,SP,NYM,371.0,24.0,100.0,93.0,92.0,86.0,0.0,Team Gorgeous
28,Justin Verlander,SP,HOU,352.0,29.0,85.0,91.0,90.0,86.0,0.0,stroke me strokeme
29,Corey Kluber,SP,CLE,347.0,30.5,84.0,90.0,80.0,93.0,0.0,Tommy's Shinebox
43,Gerrit Cole,SP,HOU,324.0,43.5,79.0,78.0,88.0,79.0,0.0,Low hanging fruit
45,Carlos Carrasco,SP,CLE,323.0,45.5,78.0,80.0,79.0,86.0,0.0,stroke me strokeme
57,Luis Severino,SP,NYY,312.0,57.5,76.0,78.0,72.0,86.0,0.0,NaN
59,Blake Snell,SP,TB,311.0,59.5,82.0,71.0,79.0,79.0,0.0,Regular Rubens
61,Aaron Nola,SP,PHI,310.0,62.5,83.0,79.0,77.0,71.0,0.0,Walk Off


In [367]:
starter[25:49]

,Player,Positions,Team,SCORE,RANK,ERAS,WHIPS,KS,WLS,SVS,Owner
158,Jose Berrios,SP,MIN,250.0,160.0,60.0,64.0,69.0,57.0,0.0,NaN
159,Masahiro Tanaka,SP,NYY,250.0,160.0,59.0,69.0,58.0,64.0,0.0,NaN
165,J.A. Happ,SP,NYY,247.0,167.0,60.0,64.0,59.0,64.0,0.0,NaN
169,Chris Archer,SP,PIT,245.0,172.0,63.0,60.0,72.0,50.0,0.0,NaN
172,Jameson Taillon,SP,PIT,245.0,172.0,67.0,66.0,62.0,50.0,0.0,Martin Awaiting Klub
174,Robbie Ray,SP,ARI,244.0,176.0,63.0,55.0,76.0,50.0,0.0,NaN
175,Mike Foltynewicz,SP,ATL,244.0,176.0,64.0,62.0,68.0,50.0,0.0,NaN
181,Zack Wheeler,SP,NYM,243.0,180.0,68.0,63.0,62.0,50.0,0.0,NaN
182,German Marquez,SP,COL,242.0,185.5,60.0,60.0,72.0,50.0,0.0,NaN
183,Rich Hill,SP,LAD,242.0,185.5,65.0,67.0,53.0,57.0,0.0,NaN


### RP

In [368]:
relief = all_players_owners[all_players_owners['Positions'].str.contains('RP')]
relief = relief[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SVS', 'Owner'] ]

In [369]:
relief[:24]

,Player,Positions,Team,SCORE,RANK,ERAS,WHIPS,KS,WLS,SVS,Owner
77,Edwin Diaz,RP,NYM,297.0,78.5,67.0,64.0,33.0,36.0,97.0,NaN
94,Brad Hand,RP,CLE,286.0,94.0,60.0,57.0,29.0,43.0,97.0,NaN
96,Kenley Jansen,RP,LAD,284.0,97.5,62.0,62.0,24.0,36.0,100.0,NaN
97,Aroldis Chapman,RP,NYY,284.0,97.5,63.0,57.0,29.0,43.0,92.0,NaN
102,Blake Treinen,RP,OAK,280.0,103.0,67.0,58.0,25.0,43.0,87.0,Foul_Tip
103,Craig Kimbrel,RP,NaN,279.0,104.0,63.0,60.0,30.0,50.0,76.0,NaN
123,Felipe Vazquez,RP,PIT,269.0,123.5,63.0,56.0,25.0,36.0,89.0,NaN
126,Roberto Osuna,RP,HOU,267.0,127.0,59.0,59.0,18.0,36.0,95.0,NaN
128,Sean Doolittle,RP,WSH,267.0,127.0,62.0,62.0,21.0,43.0,79.0,NaN
151,Wade Davis,RP,COL,254.0,152.5,54.0,53.0,22.0,36.0,89.0,NaN


## My Team

In [381]:
my_team = all_players_owners[ all_players_owners['Owner'] == 'Tommy\'s Shinebox' ]
my_team

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,ERAS,WHIPS,KS,WLS,SVS,Owner
1,Mookie Betts,"CF,RF",BOS,476.0,2.0,94.0,77.0,69.0,77.0,100.0,59.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
17,Giancarlo Stanton,"LF,RF,DH",NYY,377.0,18.0,54.0,44.0,100.0,93.0,79.0,7.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
29,Corey Kluber,SP,CLE,347.0,30.5,NaN,NaN,NaN,NaN,NaN,NaN,84.0,90.0,80.0,93.0,0.0,Tommy's Shinebox
95,Patrick Corbin,SP,WSH,285.0,96.0,NaN,NaN,NaN,NaN,NaN,NaN,74.0,68.0,72.0,71.0,0.0,Tommy's Shinebox
199,Jose Martinez,"1B,RF",STL,237.0,199.0,68.0,51.0,31.0,46.0,37.0,4.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox


In [382]:
round(my_team[ ['AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SVS'] ].mean())

AVGS     72.0
BBKOS    57.0
HRS      67.0
RBIS     72.0
RS       72.0
SBS      23.0
ERAS     79.0
WHIPS    79.0
KS       76.0
WLS      82.0
SVS       0.0
dtype: float64